In [1]:
def Main():
    Timer('Start');
    Finished = False
    Iteration = 0
    Seed = RandomSeed

    while not Finished:
        rnd.seed(Seed)
        Model = pyo.ConcreteModel(name = ModelName)
        Model.Engine = SolverName
        Model.TimeLimit = TimeLimit
        if Iteration == 0:
            print(Model.name)   # Print the model name only once
            print('Grid:    ', GridFile.rsplit('\\', 1)[-1])
            print('Lexicon: ', WordFile.rsplit('\\', 1)[-1])
        print('\nIteration: ', Iteration + 1, ' of ', MaxIterations)
        Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos = GetData(WordFile, WordWorksheet, GridFile, GridWorksheet)
        DefineModelData(Model, Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos)
        Solver, Model = SetUpSolver(Model)
        Timer('Setup');
        DefineModel(Model)
        WriteModelToFile(WriteFile, Model)   # Write model to file, if required
        Timer('Create model');
        print('Calling solver...')
        Results, Model = CallSolver(Solver, Model)
        Timer('Solved');
        
        if (Results.solver.status == pyo.SolverStatus.ok) or (Solver._solver_model.SolCount >= 1):   # Feasible/optimal or found at least one feasible solution
            print('Solution for random seed:', Seed, '\n')
            Model.solutions.load_from(Results)   # Defer loading results until now, in case there is no solution to load
            WriteOutput(Model, Results)
            if StopOnFirst:   # Ignore iteration count and stop on first solution
                Finished = True
        else:
            print('No solution for random seed:', Seed, '(', Results.solver.termination_condition, ')', '\n')
        Iteration += 1        
        Seed += 1
        if Iteration >= MaxIterations:   # If not stopping on first iteration, then stop after specified number of iterations
            Finished = True
    Timer('Finish')
    WriteCheckpoints()